In [57]:
from datasets import load_dataset
import json

from openai_service import get_chat_completion
from prompts import *
import pandas as pd
import json
from ast import literal_eval

In [100]:
# dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge")
# df = dataset['train'].to_pandas()

# df.head()

In [99]:
# df.info()

In [98]:
# type(df.iloc[0].choices['text'])

In [121]:
# df['options'] = df['choices'].apply(lambda x: x['text'].tolist())
# df.head()

In [18]:
# dataset_wino = load_dataset("winogrande", "winogrande_debiased")
# df_wino = dataset_wino['train'].to_pandas()

# df_wino.head()

In [123]:
# df.to_csv('datasets/arc_challenge_train.csv', index=False)
# df_wino.to_csv('datasets/winogrande_train.csv', index=False)

In [124]:
df_arc = pd.read_csv('datasets/arc_challenge_train.csv')
df_wino = pd.read_csv('datasets/winogrande_train.csv')

In [133]:
df_wino.rename(columns={'sentence': 'question'}, inplace=True)

In [134]:
df_wino.head()

,question,option1,option2,answer
0,John moved the couch from the garage to the ba...,garage,backyard,1
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2


In [135]:
df_arc.head()

,id,question,choices,answerKey,options
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with..."
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr..."
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c..."
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin..."
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']"


In [136]:
print(concept_guided_reasoning_prompt)

Question: - {question}
Options: - {options}

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept to the other concepts in the context of the sentence in a structured manner along with one line description for each are:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"description": "one line description"
	},
	{
		"concept": "concept2",
		"relationship": "relation",
		"related_concept": "concept3",
		"description": "one line description"
	

In [137]:
df_arc.head()

,id,question,choices,answerKey,options
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with..."
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr..."
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c..."
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin..."
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']"


In [142]:
def build_prompt_for_sample(row, name):
    question = row['question']
    if name == 'arc':
        options = str(literal_eval(row.options))
    else:
        options = str([row['option1'], row['option2']])
    prompt = concept_guided_reasoning_prompt.replace("{question}", question).replace("{options}", options)
    return prompt

In [143]:
print(build_prompt_for_sample(df_arc.iloc[0], name='arc'))

Question: - George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?
Options: - ['dry palms', 'wet palms', 'palms covered with oil', 'palms covered with lotion']

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept to the other concepts in the context of the sentence in a structured manner along with one line description for each are:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"descripti

In [144]:
print(build_prompt_for_sample(df_wino.iloc[0], name='wino'))

Question: - John moved the couch from the garage to the backyard to create space. The _ is small.
Options: - ['garage', 'backyard']

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept to the other concepts in the context of the sentence in a structured manner along with one line description for each are:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"description": "one line description"
	},
	{
		"concept": "concept2",
		"relationship"

In [141]:
len(df_arc)

1119

In [146]:
df_arc['prompt'] = df_arc.apply(lambda row: build_prompt_for_sample(row, name='arc'), axis=1)
df_arc.head()

,id,question,choices,answerKey,options,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...


In [147]:
df_arc.to_csv('datasets/arc_challenge_train_with_prompt.csv', index=False)

In [148]:
df_wino['prompt'] = df_wino.apply(lambda row: build_prompt_for_sample(row, name='wino'), axis=1)
df_wino.head()

,question,option1,option2,answer,prompt
0,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...


In [149]:
df_wino.to_csv('datasets/winogrande_train_with_prompt.csv', index=False)